## Converting California to S2 Cells

Source: https://cecgis-caenergy.opendata.arcgis.com/datasets/california-electric-transmission-line

In [159]:
# increase cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [160]:
import gc
import time
import re
import s2_py as s2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shapefile as shp
import geopandas as gpd
import pandas_profiling as pp
from shapely.geometry import Polygon, mapping, box
from datetime import date, timedelta

In [161]:
def create_S2_loop(max_poly):
    """Converts Polygon into S2 Loop"""
    points = []
    for coord in tuple(reversed(max_poly)):
        long, lat = coord
        latlng = s2.S2LatLng.FromDegrees(lat, long)
        points.append(latlng.ToPoint())
    return s2.S2Loop(points)

In [162]:
def create_S2_coverer(region, lvl):
    """Generates a list of S2 Cells of specified level"""
    coverer = s2.S2RegionCoverer()
    coverer.set_min_level(lvl)
    coverer.set_max_level(lvl)
    return coverer.GetCovering(region)

In [163]:
def split_data_frame_list(df, target_column, row_id):
    """
    Splits a column with lists into rows
    
    Arguments:
        df: dataframe
        target_column: name of column that contains lists        
        row_id: column to merge back on
    
    Returns:
        Dataframe
    """
    
    # create a new dataframe with each item in a seperate column, dropping rows with missing values
    col_df = pd.DataFrame(df[target_column].tolist())\
                .join(df[[target_column, row_id]])\
                .drop(columns=[target_column])\
                .set_index(row_id)

    # create a series with columns stacked as rows         
    stacked = col_df.stack()\
                    .reset_index()\
                    .drop(columns='level_1')
    stacked.columns = [row_id, target_column]

    return stacked

In [164]:
tl_df = gpd.read_file("./Data/Transmission_Line/Transmission_Line.shp")
tl_df.head()

,OBJECTID,Name,kV,kV_Sort,Owner,Status,Circuit,Type,Legend,Length_Mil,Length_Fee,Comments,Shape__Len,geometry
0,2001,PG&E 60kV,60,60.0,PG&E,Operational,Single,OH,PG&E_60_70kV,1.0,5813.64639807,None,0.018523,LINESTRING (-122.025217180817 39.7560586198882...
1,2002,PG&E 60kV,60,60.0,PG&E,Operational,Single,OH,PG&E_60_70kV,2.0,11459.33011218,None,0.032482,LINESTRING (-122.010209405169 39.5827809214517...
2,2003,PG&E 60kV,60,60.0,PG&E,Operational,Single,OH,PG&E_60_70kV,11.0,56984.89118592,None,0.159223,LINESTRING (-122.003238381708 39.5533528274813...
3,2004,PG&E 60kV,60,60.0,PG&E,Operational,Single,OH,PG&E_60_70kV,15.0,81003.57166241,None,0.250896,LINESTRING (-122.013588113745 39.4023799821862...
4,2005,PG&E 60kV,60,60.0,PG&E,Operational,Single,OH,PG&E_60_70kV,20.0,105641.41726686,None,0.300362,"LINESTRING (-122.153286201514 39.276960876995,..."


In [165]:
tl_df.shape

(6841, 14)

In [166]:
tl_df[list(tl_df)[:-1]].describe(include = 'all')

,OBJECTID,Name,kV,kV_Sort,Owner,Status,Circuit,Type,Legend,Length_Mil,Length_Fee,Comments,Shape__Len
count,6841.000000,6841,6817,6817.000000,6817,6817,6817,6817,6817,6793.000000,6804,581,6.841000e+03
unique,NaN,99,21,NaN,36,3,6,4,35,NaN,6782,156,NaN
top,NaN,SCE 66kV,66,NaN,PG&E,Operational,Single,OH,SCE_33_69kV,NaN,478.71883579,Partially underground,NaN
freq,NaN,1463,1498,NaN,2826,6802,5437,6696,1646,NaN,2,181,NaN
mean,3421.000000,NaN,NaN,116.437949,NaN,NaN,NaN,NaN,NaN,5.156485,NaN,NaN,8.511326e-02
std,1974.970928,NaN,NaN,84.262934,NaN,NaN,NaN,NaN,NaN,12.254171,NaN,NaN,1.948325e-01
min,1.000000,NaN,NaN,33.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,2.802626e-08
25%,1711.000000,NaN,NaN,66.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,5.894882e-03
50%,3421.000000,NaN,NaN,69.000000,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,2.769896e-02
75%,5131.000000,NaN,NaN,115.000000,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,NaN,8.373453e-02


In [167]:
pp.ProfileReport(tl_df[list(tl_df)[:-1]])

Number of variables,13
Number of observations,6841
Total Missing (%),7.3%
Total size in memory,694.9 KiB
Average record size in memory,104.0 B
Numeric,3
Categorical,9
Boolean,0
Date,0
Text (Unique),0
Rejected,1


In [168]:
len(mapping(tl_df.geometry[121])['coordinates'])

3

In [169]:
def create_S2_polyline(linestring):
    polylines = []
    mapping_dict = mapping(linestring)
    coords = mapping_dict['coordinates']
    if mapping_dict['type']=='LineString':
        points = []
        for coord in coords:
            long, lat = coord
            latlng = s2.S2LatLng.FromDegrees(lat, long)
            points.append(latlng)        
        polylines.append(s2.S2Polyline(points))
    elif mapping_dict['type']=='MultiLineString':
        for coord in coords:
            points = []
            for subcoord in coord:
                long, lat = subcoord
                latlng = s2.S2LatLng.FromDegrees(lat, long)
                points.append(latlng)
            polylines.append(s2.S2Polyline(points))
    return polylines

In [170]:
tl_df['S2_PolyLine'] = tl_df.geometry.apply(create_S2_polyline)
tl_df.head()

,OBJECTID,Name,kV,kV_Sort,Owner,Status,Circuit,Type,Legend,Length_Mil,Length_Fee,Comments,Shape__Len,geometry,S2_PolyLine
0,2001,PG&E 60kV,60,60.0,PG&E,Operational,Single,OH,PG&E_60_70kV,1.0,5813.64639807,None,0.018523,LINESTRING (-122.025217180817 39.7560586198882...,[<s2_py.pywraps2.S2Polyline; proxy of <Swig Ob...
1,2002,PG&E 60kV,60,60.0,PG&E,Operational,Single,OH,PG&E_60_70kV,2.0,11459.33011218,None,0.032482,LINESTRING (-122.010209405169 39.5827809214517...,[<s2_py.pywraps2.S2Polyline; proxy of <Swig Ob...
2,2003,PG&E 60kV,60,60.0,PG&E,Operational,Single,OH,PG&E_60_70kV,11.0,56984.89118592,None,0.159223,LINESTRING (-122.003238381708 39.5533528274813...,[<s2_py.pywraps2.S2Polyline; proxy of <Swig Ob...
3,2004,PG&E 60kV,60,60.0,PG&E,Operational,Single,OH,PG&E_60_70kV,15.0,81003.57166241,None,0.250896,LINESTRING (-122.013588113745 39.4023799821862...,[<s2_py.pywraps2.S2Polyline; proxy of <Swig Ob...
4,2005,PG&E 60kV,60,60.0,PG&E,Operational,Single,OH,PG&E_60_70kV,20.0,105641.41726686,None,0.300362,"LINESTRING (-122.153286201514 39.276960876995,...",[<s2_py.pywraps2.S2Polyline; proxy of <Swig Ob...


In [171]:
tl_df.OBJECTID.nunique()

6841

In [172]:
tl2_df = tl_df[['OBJECTID', 'S2_PolyLine']]
tl2_df = split_data_frame_list(tl2_df, 'S2_PolyLine', 'OBJECTID')
tl2_df.shape

(6957, 2)

In [173]:
tl2_df.head()

,OBJECTID,S2_PolyLine
0,2001,<s2_py.pywraps2.S2Polyline; proxy of <Swig Obj...
1,2002,<s2_py.pywraps2.S2Polyline; proxy of <Swig Obj...
2,2003,<s2_py.pywraps2.S2Polyline; proxy of <Swig Obj...
3,2004,<s2_py.pywraps2.S2Polyline; proxy of <Swig Obj...
4,2005,<s2_py.pywraps2.S2Polyline; proxy of <Swig Obj...


In [174]:
tl2_df['S2_Cells_low'] = tl2_df.S2_PolyLine.apply(create_S2_coverer, args=[9])
tl2_df['S2_Cells_med'] = tl2_df.S2_PolyLine.apply(create_S2_coverer, args=[10])
tl2_df['S2_Cells_high'] = tl2_df.S2_PolyLine.apply(create_S2_coverer, args=[11])
tl2_df.shape

(6957, 5)

In [175]:
tl2_df.head()

,OBJECTID,S2_PolyLine,S2_Cells_low,S2_Cells_med,S2_Cells_high
0,2001,<s2_py.pywraps2.S2Polyline; proxy of <Swig Obj...,"(4/001001130 ,)","(4/0010011300 ,)","(4/00100113001 ,)"
1,2002,<s2_py.pywraps2.S2Polyline; proxy of <Swig Obj...,"(4/001001202 , 4/001001203 )","(4/0010012023 , 4/0010012030 )","(4/00100120233 , 4/00100120300 )"
2,2003,<s2_py.pywraps2.S2Polyline; proxy of <Swig Obj...,"(4/001001202 , 4/001001231 )","(4/0010012022 , 4/0010012023 , 4/0010012311 )","(4/00100120222 , 4/00100120223 , 4/00100120230..."
3,2004,<s2_py.pywraps2.S2Polyline; proxy of <Swig Obj...,"(4/001001231 ,)","(4/0010012310 , 4/0010012311 , 4/0010012313 )","(4/00100123100 , 4/00100123101 , 4/00100123102..."
4,2005,<s2_py.pywraps2.S2Polyline; proxy of <Swig Obj...,"(4/001001200 , 4/001001201 , 4/001001231 , 4/0...","(4/0010012002 , 4/0010012012 , 4/0010012013 , ...","(4/00100120022 , 4/00100120122 , 4/00100120123..."


In [176]:
tl_low_df = tl2_df[['OBJECTID', 'S2_Cells_low']]
tl_low_df = split_data_frame_list(tl_low_df, 'S2_Cells_low', 'OBJECTID')
tl_low_df['S2_Cells_ID'] = tl_low_df.S2_Cells_low.apply(lambda x: x.ToToken())
## Some cells are duplicated mostly because there are usually multiple polygons per wildfire
tl_low_df = tl_low_df.drop_duplicates(subset=['OBJECTID', 'S2_Cells_ID']).reset_index().drop(columns='index')
tl_low_df['FZ_grp'] = 'low'
tl_low_df.shape

(10939, 4)

In [177]:
tl_low_df.head()

,OBJECTID,S2_Cells_low,S2_Cells_ID,FZ_grp
0,2001,4/001001130 ,8082e4,low
1,2002,4/001001202 ,808314,low
2,2002,4/001001203 ,80831c,low
3,2003,4/001001202 ,808314,low
4,2003,4/001001231 ,80836c,low


In [178]:
tl_low_df.S2_Cells_ID.nunique()

1060

In [179]:
tl_med_df = tl2_df[['OBJECTID', 'S2_Cells_med']]
tl_med_df = split_data_frame_list(tl_med_df, 'S2_Cells_med', 'OBJECTID')
tl_med_df['S2_Cells_ID'] = tl_med_df.S2_Cells_med.apply(lambda x: x.ToToken())
## Some cells are duplicated mostly because there are usually multiple polygons per wildfire
tl_med_df = tl_med_df.drop_duplicates(subset=['OBJECTID', 'S2_Cells_ID']).reset_index().drop(columns='index')
tl_med_df['FZ_grp'] = 'med'
tl_med_df.shape

(15192, 4)

In [180]:
tl_med_df.head()

,OBJECTID,S2_Cells_med,S2_Cells_ID,FZ_grp
0,2001,4/0010011300 ,8082e1,med
1,2002,4/0010012023 ,808317,med
2,2002,4/0010012030 ,808319,med
3,2003,4/0010012022 ,808315,med
4,2003,4/0010012023 ,808317,med


In [181]:
tl_med_df.S2_Cells_ID.nunique()

2943

In [182]:
tl_high_df = tl2_df[['OBJECTID', 'S2_Cells_high']]
tl_high_df = split_data_frame_list(tl_high_df, 'S2_Cells_high', 'OBJECTID')
tl_high_df['S2_Cells_ID'] = tl_high_df.S2_Cells_high.apply(lambda x: x.ToToken())
## Some cells are duplicated mostly because there are usually multiple polygons per wildfire
tl_high_df = tl_high_df.drop_duplicates(subset=['OBJECTID', 'S2_Cells_ID']).reset_index().drop(columns='index')
tl_high_df['FZ_grp'] = 'high'
tl_high_df.shape

(23559, 4)

In [183]:
tl_high_df.head()

,OBJECTID,S2_Cells_high,S2_Cells_ID,FZ_grp
0,2001,4/00100113001 ,8082e0c,high
1,2002,4/00100120233 ,808317c,high
2,2002,4/00100120300 ,8083184,high
3,2003,4/00100120222 ,8083154,high
4,2003,4/00100120223 ,808315c,high


In [184]:
tl_high_df.S2_Cells_ID.nunique()

7677

In [185]:
list(tl_df)[:-2]

['OBJECTID',
 'Name',
 'kV',
 'kV_Sort',
 'Owner',
 'Status',
 'Circuit',
 'Type',
 'Legend',
 'Length_Mil',
 'Length_Fee',
 'Comments',
 'Shape__Len']

In [186]:
tl_low_df.shape[0]+tl_med_df.shape[0]+tl_high_df.shape[0]

49690

In [187]:
tl_s2_stacked_df = tl_low_df.drop(columns='S2_Cells_low').append(tl_med_df.drop(columns='S2_Cells_med')).append(tl_high_df.drop(columns='S2_Cells_high')).reset_index().drop(columns='index')
tl_s2_stacked_df = pd.merge(tl_s2_stacked_df, tl_df[list(tl_df)[:-2]], on='OBJECTID', how='left')
tl_s2_stacked_df.shape

(49690, 15)

In [188]:
tl_s2_stacked_df.head()

,OBJECTID,S2_Cells_ID,FZ_grp,Name,kV,kV_Sort,Owner,Status,Circuit,Type,Legend,Length_Mil,Length_Fee,Comments,Shape__Len
0,2001,8082e4,low,PG&E 60kV,60,60.0,PG&E,Operational,Single,OH,PG&E_60_70kV,1.0,5813.64639807,None,0.018523
1,2002,808314,low,PG&E 60kV,60,60.0,PG&E,Operational,Single,OH,PG&E_60_70kV,2.0,11459.33011218,None,0.032482
2,2002,80831c,low,PG&E 60kV,60,60.0,PG&E,Operational,Single,OH,PG&E_60_70kV,2.0,11459.33011218,None,0.032482
3,2003,808314,low,PG&E 60kV,60,60.0,PG&E,Operational,Single,OH,PG&E_60_70kV,11.0,56984.89118592,None,0.159223
4,2003,80836c,low,PG&E 60kV,60,60.0,PG&E,Operational,Single,OH,PG&E_60_70kV,11.0,56984.89118592,None,0.159223


In [189]:
tl_s2_stacked_df.OBJECTID.nunique()

6841

In [190]:
tl_s2_stacked_df[tl_s2_stacked_df.duplicated(subset=['OBJECTID', 'S2_Cells_ID'])].shape

(0, 15)

In [191]:
ca_s2_df = pd.read_csv('./Data/Processed/CA_FZ_S2Cells.csv')
ca_s2_df.shape

(10643, 8)

In [192]:
tl_s2_stacked_df['TransmissionLine'] = 1
ca_s2_tl_df = pd.merge(ca_s2_df[['S2_Cells_ID']], tl_s2_stacked_df.drop(columns='FZ_grp'), on='S2_Cells_ID', how='inner')
# ca_s2_tl_df.TransmissionLine = ca_s2_tl_df.TransmissionLine.fillna(0)
ca_s2_tl_df.shape

(19986, 15)

In [193]:
ca_s2_tl_df.head()

,S2_Cells_ID,OBJECTID,Name,kV,kV_Sort,Owner,Status,Circuit,Type,Legend,Length_Mil,Length_Fee,Comments,Shape__Len,TransmissionLine
0,80be44,5462,SCE 55kV,55,55.0,SCE,Operational,Single,OH,SCE_33_69kV,33.0,174178.77181368,Unsured the path for lines as the aerial photo...,0.567475,1
1,80be44,5463,SCE 55kV,55,55.0,SCE,Operational,Single,OH,SCE_33_69kV,17.0,89549.42235258,Unsured the path for lines as the aerial photo...,0.285736,1
2,80be44,5464,SCE 55kV,55,55.0,SCE,Operational,Single,OH,SCE_33_69kV,0.0,2081.67093486,Unsured the path for lines as the aerial photo...,0.006319,1
3,80be44,5465,SCE 55kV,55,55.0,SCE,Operational,Single,OH,SCE_33_69kV,10.0,51151.79497193,Unsured the path for lines as the aerial photo...,0.171424,1
4,80be4c,3351,SCE 115kV,115,115.0,SCE,Operational,Single,OH,SCE_115_161kV,126.0,666661.95337937,None,1.893312,1


In [194]:
ca_s2_tl_df.S2_Cells_ID.nunique()

4188

In [195]:
ca_s2_tl_df.OBJECTID.nunique(), ca_s2_tl_df.Name.nunique()

(6840, 98)

In [196]:
ca_s2_tl_df.TransmissionLine.sum()

19986

In [197]:
ca_s2_tl_df.Length_Fee = pd.to_numeric(ca_s2_tl_df.Length_Fee)

In [198]:
ca_s2_tl_df.Length_Fee.isna().sum()

217

In [199]:
ca_s2_tl_df[['Length_Fee', 'Length_Mil']].corr()

,Length_Fee,Length_Mil
Length_Fee,1.00000,0.99997
Length_Mil,0.99997,1.00000


In [200]:
ca_s2_tl_df[['Shape__Len', 'Length_Mil']].corr()

,Shape__Len,Length_Mil
Shape__Len,1.000000,0.993878
Length_Mil,0.993878,1.000000


In [201]:
ca_s2_tl_df.Name.nunique()

98

In [202]:
ca_s2_tl_df.kV.nunique()

21

In [203]:
ca_s2_tl_df.kV_Sort.nunique()

20

In [204]:
ca_s2_tl_df.Owner.nunique()

36

In [205]:
owner = pd.get_dummies(ca_s2_tl_df.Owner)
owner.columns = ['OWNER_'+i for i in owner.columns]
owner.head()

,OWNER_AMP,OWNER_ANZA,OWNER_APUD,OWNER_BLYTHE ENERGY,OWNER_BPA,OWNER_BVES,OWNER_CALPECO,OWNER_CCSF,OWNER_ESJ,OWNER_IID,...,OWNER_SMUD,OWNER_SVEC,OWNER_SVP,OWNER_TANC,OWNER_TID,OWNER_TPUD,OWNER_TRANS BAY CABLE,OWNER_VENON,OWNER_VICTORVILLE,OWNER_WAPA
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [206]:
ca_s2_tl_df.Status.value_counts()

Operational    19726
Proposed          48
Closed             2
Name: Status, dtype: int64

In [207]:
ca_s2_tl_df.Circuit.value_counts()

Single            15230
Double             4440
Many                 95
Liberty Energy        6
Duble                 4
Quad                  1
Name: Circuit, dtype: int64

In [208]:
circuit = pd.get_dummies(ca_s2_tl_df.Circuit)
circuit.columns = ['CIRCUIT_'+i for i in circuit.columns]
circuit.head()

,CIRCUIT_Double,CIRCUIT_Duble,CIRCUIT_Liberty Energy,CIRCUIT_Many,CIRCUIT_Quad,CIRCUIT_Single
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,0,0,0,0,0,1


In [209]:
ca_s2_tl_df.Type.value_counts()

OH    19507
UG      251
UW       17
ug        1
Name: Type, dtype: int64

In [210]:
typex = pd.get_dummies(ca_s2_tl_df.Type)
typex.columns = ['TYPE_'+i for i in typex.columns]
typex.TYPE_UG = typex.TYPE_UG + typex.TYPE_ug
typex.drop(columns='TYPE_ug', inplace=True)
typex.head()

,TYPE_OH,TYPE_UG,TYPE_UW
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [211]:
ca_s2_tl_df.Legend.nunique()

35

In [212]:
ca_s2_tl_df.Length_Mil.describe()

count    19677.000000
mean        20.223103
std         35.998779
min          0.000000
25%          2.000000
50%          7.000000
75%         21.000000
max        294.000000
Name: Length_Mil, dtype: float64

In [213]:
ca_s2_tl_df.Comments.nunique()

156

In [214]:
ca_s2_tl_df.head()

,S2_Cells_ID,OBJECTID,Name,kV,kV_Sort,Owner,Status,Circuit,Type,Legend,Length_Mil,Length_Fee,Comments,Shape__Len,TransmissionLine
0,80be44,5462,SCE 55kV,55,55.0,SCE,Operational,Single,OH,SCE_33_69kV,33.0,174178.771814,Unsured the path for lines as the aerial photo...,0.567475,1
1,80be44,5463,SCE 55kV,55,55.0,SCE,Operational,Single,OH,SCE_33_69kV,17.0,89549.422353,Unsured the path for lines as the aerial photo...,0.285736,1
2,80be44,5464,SCE 55kV,55,55.0,SCE,Operational,Single,OH,SCE_33_69kV,0.0,2081.670935,Unsured the path for lines as the aerial photo...,0.006319,1
3,80be44,5465,SCE 55kV,55,55.0,SCE,Operational,Single,OH,SCE_33_69kV,10.0,51151.794972,Unsured the path for lines as the aerial photo...,0.171424,1
4,80be4c,3351,SCE 115kV,115,115.0,SCE,Operational,Single,OH,SCE_115_161kV,126.0,666661.953379,None,1.893312,1


In [215]:
ca_s2_tl_dummies_df = ca_s2_tl_df.join(owner).join(circuit).join(typex)
ca_s2_tl_dummies_df.shape

(19986, 60)

In [216]:
ca_s2_tl_dummies_df.head()

,S2_Cells_ID,OBJECTID,Name,kV,kV_Sort,Owner,Status,Circuit,Type,Legend,...,OWNER_WAPA,CIRCUIT_Double,CIRCUIT_Duble,CIRCUIT_Liberty Energy,CIRCUIT_Many,CIRCUIT_Quad,CIRCUIT_Single,TYPE_OH,TYPE_UG,TYPE_UW
0,80be44,5462,SCE 55kV,55,55.0,SCE,Operational,Single,OH,SCE_33_69kV,...,0,0,0,0,0,0,1,1,0,0
1,80be44,5463,SCE 55kV,55,55.0,SCE,Operational,Single,OH,SCE_33_69kV,...,0,0,0,0,0,0,1,1,0,0
2,80be44,5464,SCE 55kV,55,55.0,SCE,Operational,Single,OH,SCE_33_69kV,...,0,0,0,0,0,0,1,1,0,0
3,80be44,5465,SCE 55kV,55,55.0,SCE,Operational,Single,OH,SCE_33_69kV,...,0,0,0,0,0,0,1,1,0,0
4,80be4c,3351,SCE 115kV,115,115.0,SCE,Operational,Single,OH,SCE_115_161kV,...,0,0,0,0,0,0,1,1,0,0


In [217]:
ca_s2_tl_grp1_df = ca_s2_tl_df.groupby('S2_Cells_ID').agg({'OBJECTID': 'nunique', 'kV_Sort': ['sum', 'mean', 'max'], 'Owner': 'nunique', 'Circuit': 'nunique', 'Length_Mil': ['sum', 'mean', 'max']}).reset_index()
ca_s2_tl_grp1_df.columns = ['S2_Cells_ID'] + [i+"_"+j for i, j in ca_s2_tl_grp1_df.columns[1:]]
ca_s2_tl_grp1_df.shape

(4188, 10)

In [218]:
ca_s2_tl_grp1_df.head()

,S2_Cells_ID,OBJECTID_nunique,kV_Sort_sum,kV_Sort_mean,kV_Sort_max,Owner_nunique,Circuit_nunique,Length_Mil_sum,Length_Mil_mean,Length_Mil_max
0,54c934,4,1138.0,284.5,500.0,2,1,161.0,40.25,149.0
1,54c935c,2,138.0,69.0,69.0,1,1,5.0,2.50,3.0
2,54c9364,2,138.0,69.0,69.0,1,1,5.0,2.50,3.0
3,54c9484,5,2230.0,446.0,500.0,4,1,411.0,82.20,149.0
4,54c948c,3,1230.0,410.0,500.0,3,1,255.0,85.00,94.0


In [219]:
ca_s2_tl_grp2_df = ca_s2_tl_dummies_df[['S2_Cells_ID']+list(ca_s2_tl_dummies_df)[15:]].groupby('S2_Cells_ID').sum().reset_index()
ca_s2_tl_grp2_df.shape

(4188, 46)

In [220]:
ca_s2_tl_grp2_df.head()

,S2_Cells_ID,OWNER_AMP,OWNER_ANZA,OWNER_APUD,OWNER_BLYTHE ENERGY,OWNER_BPA,OWNER_BVES,OWNER_CALPECO,OWNER_CCSF,OWNER_ESJ,...,OWNER_WAPA,CIRCUIT_Double,CIRCUIT_Duble,CIRCUIT_Liberty Energy,CIRCUIT_Many,CIRCUIT_Quad,CIRCUIT_Single,TYPE_OH,TYPE_UG,TYPE_UW
0,54c934,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4,4,0,0
1,54c935c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,2,0,0
2,54c9364,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,2,0,0
3,54c9484,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,5,5,0,0
4,54c948c,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,3,3,0,0


In [221]:
ca_s2_tl_final_df = pd.merge(ca_s2_tl_grp1_df, ca_s2_tl_grp2_df, on='S2_Cells_ID', how='left')
ca_s2_tl_final_df.shape

(4188, 55)

In [222]:
tl2_df['S2_Cells_lvl16'] = tl2_df.S2_PolyLine.apply(create_S2_coverer, args=[16])
tl_lvl16_df = tl2_df[['OBJECTID', 'S2_Cells_lvl16']]
tl_lvl16_df = split_data_frame_list(tl_lvl16_df, 'S2_Cells_lvl16', 'OBJECTID')
tl_lvl16_df['S2_Cells_ID'] = tl_lvl16_df.S2_Cells_lvl16.apply(lambda x: x.ToToken())
## Some cells are duplicated mostly because there are usually multiple polygons per wildfire
tl_lvl16_df = tl_lvl16_df.drop_duplicates(subset=['OBJECTID', 'S2_Cells_ID']).reset_index().drop(columns='index')
tl_lvl16_df['S2_Cells_ID_lvl9'] = tl_lvl16_df.S2_Cells_lvl16.apply(lambda x: x.parent(9).ToToken())
tl_lvl16_df['S2_Cells_ID_lvl10'] = tl_lvl16_df.S2_Cells_lvl16.apply(lambda x: x.parent(10).ToToken())
tl_lvl16_df['S2_Cells_ID_lvl11'] = tl_lvl16_df.S2_Cells_lvl16.apply(lambda x: x.parent(11).ToToken())
tl_lvl16_df.shape

(559823, 6)

In [223]:
tl_lvl16_df.head()

,OBJECTID,S2_Cells_lvl16,S2_Cells_ID,S2_Cells_ID_lvl9,S2_Cells_ID_lvl10,S2_Cells_ID_lvl11
0,2001,4/0010011300101202 ,8082e08c5,8082e4,8082e1,8082e0c
1,2001,4/0010011300101203 ,8082e08c7,8082e4,8082e1,8082e0c
2,2001,4/0010011300101213 ,8082e08cf,8082e4,8082e1,8082e0c
3,2001,4/0010011300101220 ,8082e08d1,8082e4,8082e1,8082e0c
4,2001,4/0010011300101222 ,8082e08d5,8082e4,8082e1,8082e0c


In [224]:
tl_lvl16_df.S2_Cells_ID.nunique()

427994

In [225]:
tl_low_grp_df = pd.DataFrame(tl_lvl16_df.groupby('S2_Cells_ID_lvl9').S2_Cells_ID.count()).reset_index()
tl_low_grp_df.columns = ['S2_Cells_ID', 'S2_Cells_lvl16_cnt']
tl_low_grp_df['S2_Cells_lvl16_cnt_coverage'] = tl_low_grp_df.S2_Cells_lvl16_cnt / (4**7)
tl_low_grp_df.head()

,S2_Cells_ID,S2_Cells_lvl16_cnt,S2_Cells_lvl16_cnt_coverage
0,54c934,251,0.015320
1,54c94c,1086,0.066284
2,54c954,427,0.026062
3,54ca2c,207,0.012634
4,54ca34,275,0.016785


In [226]:
tl_low_grp_df.S2_Cells_lvl16_cnt_coverage.describe()

count    1060.000000
mean        0.032235
std         0.034082
min         0.000061
25%         0.009583
50%         0.021088
75%         0.043518
max         0.268616
Name: S2_Cells_lvl16_cnt_coverage, dtype: float64

In [227]:
tl_med_grp_df = pd.DataFrame(tl_lvl16_df.groupby('S2_Cells_ID_lvl10').S2_Cells_ID.count()).reset_index()
tl_med_grp_df.columns = ['S2_Cells_ID', 'S2_Cells_lvl16_cnt']
tl_med_grp_df['S2_Cells_lvl16_cnt_coverage'] = tl_med_grp_df.S2_Cells_lvl16_cnt / (4**6)
tl_med_grp_df.head()

,S2_Cells_ID,S2_Cells_lvl16_cnt,S2_Cells_lvl16_cnt_coverage
0,54c931,29,0.007080
1,54c935,20,0.004883
2,54c937,202,0.049316
3,54c949,416,0.101562
4,54c94b,182,0.044434


In [228]:
tl_med_grp_df.S2_Cells_lvl16_cnt_coverage.describe()

count    2943.000000
mean        0.046441
std         0.045826
min         0.000244
25%         0.017822
50%         0.032227
75%         0.060669
max         0.401611
Name: S2_Cells_lvl16_cnt_coverage, dtype: float64

In [229]:
tl_high_grp_df = pd.DataFrame(tl_lvl16_df.groupby('S2_Cells_ID_lvl11').S2_Cells_ID.count()).reset_index()
tl_high_grp_df.columns = ['S2_Cells_ID', 'S2_Cells_lvl16_cnt']
tl_high_grp_df['S2_Cells_lvl16_cnt_coverage'] = tl_high_grp_df.S2_Cells_lvl16_cnt / (4**5)
tl_high_grp_df.head()

,S2_Cells_ID,S2_Cells_lvl16_cnt,S2_Cells_lvl16_cnt_coverage
0,54c930c,29,0.028320
1,54c935c,20,0.019531
2,54c9364,33,0.032227
3,54c9374,78,0.076172
4,54c937c,91,0.088867


In [230]:
tl_high_grp_df.S2_Cells_lvl16_cnt_coverage.describe()

count    7677.000000
mean        0.071213
std         0.066728
min         0.000977
25%         0.032227
50%         0.048828
75%         0.091797
max         0.669922
Name: S2_Cells_lvl16_cnt_coverage, dtype: float64

In [231]:
tl_low_grp_df.shape, tl_med_grp_df.shape, tl_high_grp_df.shape

((1060, 3), (2943, 3), (7677, 3))

In [232]:
tl_low_grp_df.head()

,S2_Cells_ID,S2_Cells_lvl16_cnt,S2_Cells_lvl16_cnt_coverage
0,54c934,251,0.015320
1,54c94c,1086,0.066284
2,54c954,427,0.026062
3,54ca2c,207,0.012634
4,54ca34,275,0.016785


In [233]:
tl_s2_grp_stacked_df = tl_low_grp_df.append(tl_med_grp_df).append(tl_high_grp_df).reset_index().drop(columns='index')
ca_s2_tl_final_df = pd.merge(ca_s2_tl_final_df, tl_s2_grp_stacked_df, on='S2_Cells_ID', how='inner')
ca_s2_tl_final_df.columns = ['S2_Cells_ID'] + ['TL_'+i for i in ca_s2_tl_final_df.columns if i != 'S2_Cells_ID']
ca_s2_tl_final_df.shape

(4188, 57)

In [234]:
ca_s2_tl_final_df.head().T

,0,1,2,3,4
S2_Cells_ID,54c934,54c935c,54c9364,54c9484,54c948c
TL_OBJECTID_nunique,4,2,2,5,3
TL_kV_Sort_sum,1138,138,138,2230,1230
TL_kV_Sort_mean,284.5,69,69,446,410
TL_kV_Sort_max,500,69,69,500,500
TL_Owner_nunique,2,1,1,4,3
TL_Circuit_nunique,1,1,1,1,1
TL_Length_Mil_sum,161,5,5,411,255
TL_Length_Mil_mean,40.25,2.5,2.5,82.2,85
TL_Length_Mil_max,149,3,3,149,94


In [235]:
ca_s2_tl_final_df.describe().T

,count,mean,std,min,25%,50%,75%,max
TL_OBJECTID_nunique,4188.0,4.772206,8.440334,1.000000,1.000000,2.000000,5.000000,190.000000
TL_kV_Sort_sum,4188.0,732.207020,1131.151832,33.000000,120.000000,345.000000,960.000000,17177.000000
TL_kV_Sort_mean,4188.0,162.802974,123.773250,33.000000,69.000000,115.000000,220.000000,500.000000
TL_kV_Sort_max,4188.0,215.714422,165.413024,33.000000,69.000000,115.000000,230.000000,500.000000
TL_Owner_nunique,4188.0,1.237584,0.525583,1.000000,1.000000,1.000000,1.000000,5.000000
TL_Circuit_nunique,4188.0,1.290115,0.471928,1.000000,1.000000,1.000000,2.000000,3.000000
TL_Length_Mil_sum,4188.0,95.016714,125.457492,0.000000,20.000000,46.000000,128.000000,989.000000
TL_Length_Mil_mean,4187.0,30.282414,35.897747,0.000000,8.969697,16.500000,38.000000,265.000000
TL_Length_Mil_max,4187.0,46.091712,52.262465,0.000000,14.000000,26.000000,60.000000,294.000000
TL_OWNER_AMP,4188.0,0.001433,0.069099,0.000000,0.000000,0.000000,0.000000,4.000000


In [236]:
ca_s2_tl_final_df.to_csv('./Data/Processed/Transmission_Lines_S2Cells.csv', index=False)